# Machine Learning Report

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import seaborn as sns
import matplotlib.pyplot as plt
import math

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

import warnings

In [40]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14269224503645536298
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6590298194
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13653841540650502511
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


## Contents
- Loading the data
- Experimenting with the data
    - PCA
    - Regularisation
        - Using Preprocessing
        - Using MinMax Scaling
- Deep Learning Approach
- Classical Learning Approaches
    - Supervised Learning
        - K-Nearest Neighbours
        - Support Vector Machine
    - Unsupervised Learning
        - K-Means

## Loading the data

In [9]:
atlas_labels = pd.read_csv("ATLAS-labels.csv", header=None)
atlas_data = pd.read_csv("ATLAS-data.csv", header=None)

In [10]:
atlas_data.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,243.128,96.455,158.618,90.253,0.693,56.238,1.581,3.615,13.597,264.586,...,2.121,376.410,3.0,46.248,-0.958,-0.444,42.759,-1.651,-1.445,157.638
1,139.751,74.028,111.619,44.266,2.119,104.723,-0.498,2.533,9.437,168.638,...,0.447,252.698,2.0,36.798,0.269,-1.254,31.816,-1.849,0.664,68.614
2,116.217,73.136,59.239,101.850,0.625,85.539,2.979,1.922,28.049,202.636,...,1.816,271.485,2.0,82.453,2.066,-1.210,58.216,1.442,-2.288,140.669
3,64.544,0.099,28.945,341.418,2.390,358.936,-1.181,0.714,35.293,508.349,...,-2.409,470.806,3.0,331.152,-1.693,0.615,41.742,0.697,1.083,409.977
4,32.281,67.393,26.006,29.088,1.330,108.191,-0.362,0.846,48.389,152.087,...,-1.421,244.220,2.0,48.198,0.949,1.517,40.382,-0.382,-2.029,88.580


### Experimenting with the Data

#### Principal Component Analysis

In [11]:
scaler = StandardScaler()
scaler.fit(atlas_data)
atlas_processed = scaler.transform(atlas_data)

In [13]:
pca = PCA(.95)
pca.fit(atlas_processed)
atlas_processed = pca.transform(atlas_processed)
atlas_processed.shape

(10000, 17)

#### Reguralization

##### Using preprocessing

In [21]:
atlas_processed = preprocessing.scale(atlas_data)
atlas_processed.shape

(10000, 30)

##### Using MinMax Scaling

In [4]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(atlas_data)
atlas_processed = scaling.transform(atlas_data)

## Deep Learning Approach

### Neural Network

In [22]:
NN_Data = atlas_processed
NN_Labels = atlas_labels.to_numpy()

In [23]:
nn_train_values, nn_test_values, nn_train_labels, nn_test_labels = train_test_split(NN_Data, NN_Labels, test_size=0.2)

Declaring some values that can be changed to experiment with the network whilst creating.

In [32]:
learning_rate = 0.5
epochs = 1000
batch_size = 100

In [47]:
inputs=keras.Input(shape=(30,))
hidden_layer_1=layers.Dense(512,activation='relu')
hidden_layer_2=layers.Dense(256,activation='sigmoid')
hidden_layer_3=layers.Dense(128,activation='relu')
output_layer=layers.Dense(1,activation='sigmoid')

z=hidden_layer_1(inputs)
z=hidden_layer_2(z)
z=hidden_layer_3(z)
output=output_layer(z)

model=keras.Model(inputs=inputs,outputs=output)

In [48]:
model.compile(loss='mean_squared_error',metrics=['accuracy'], optimizer='adam')
model.fit(nn_train_values,nn_train_labels,epochs=epochs,verbose=1,validation_split=0.2)

Train on 6400 samples, validate on 1600 samples
Epoch 1/1000
6400/6400 [==============================] - 1s 109us/sample - loss: 0.1673 - accuracy: 0.7503 - val_loss: 0.1271 - val_accuracy: 0.8106
Epoch 2/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.1278 - accuracy: 0.8211 - val_loss: 0.1254 - val_accuracy: 0.8194
Epoch 3/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.1212 - accuracy: 0.8311 - val_loss: 0.1196 - val_accuracy: 0.8300
Epoch 4/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.1153 - accuracy: 0.8422 - val_loss: 0.1182 - val_accuracy: 0.8331
Epoch 5/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.1137 - accuracy: 0.8406 - val_loss: 0.1170 - val_accuracy: 0.8344
Epoch 6/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.1115 - accuracy: 0.8498 - val_loss: 0.1175 - val_accuracy: 0.8319
Epoch 7/1000
6400/6400 [==============================] -

6400/6400 [==============================] - 0s 45us/sample - loss: 0.0191 - accuracy: 0.9809 - val_loss: 0.1556 - val_accuracy: 0.8300
Epoch 56/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0241 - accuracy: 0.9723 - val_loss: 0.1720 - val_accuracy: 0.8100
Epoch 57/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0284 - accuracy: 0.9686 - val_loss: 0.1634 - val_accuracy: 0.8150
Epoch 58/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0297 - accuracy: 0.9658 - val_loss: 0.1571 - val_accuracy: 0.8231
Epoch 59/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0233 - accuracy: 0.9750 - val_loss: 0.1625 - val_accuracy: 0.8163
Epoch 60/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0236 - accuracy: 0.9734 - val_loss: 0.1559 - val_accuracy: 0.8281
Epoch 61/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0225 - accuracy: 0.9756 - val_

6400/6400 [==============================] - 0s 46us/sample - loss: 0.0153 - accuracy: 0.9847 - val_loss: 0.1654 - val_accuracy: 0.8194
Epoch 110/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0159 - accuracy: 0.9839 - val_loss: 0.1655 - val_accuracy: 0.8175
Epoch 111/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0175 - accuracy: 0.9814 - val_loss: 0.1666 - val_accuracy: 0.8219
Epoch 112/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0195 - accuracy: 0.9789 - val_loss: 0.1657 - val_accuracy: 0.8200
Epoch 113/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0193 - accuracy: 0.9795 - val_loss: 0.1689 - val_accuracy: 0.8156
Epoch 114/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0171 - accuracy: 0.9820 - val_loss: 0.1624 - val_accuracy: 0.8219
Epoch 115/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0179 - accuracy: 0.9811 

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0195 - accuracy: 0.9784 - val_loss: 0.1656 - val_accuracy: 0.8194
Epoch 164/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0165 - accuracy: 0.9827 - val_loss: 0.1614 - val_accuracy: 0.8219
Epoch 165/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0146 - accuracy: 0.9856 - val_loss: 0.1660 - val_accuracy: 0.8138
Epoch 166/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0145 - accuracy: 0.9852 - val_loss: 0.1621 - val_accuracy: 0.8206
Epoch 167/1000
6400/6400 [==============================] - 0s 55us/sample - loss: 0.0147 - accuracy: 0.9850 - val_loss: 0.1650 - val_accuracy: 0.8188
Epoch 168/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0138 - accuracy: 0.9862 - val_loss: 0.1641 - val_accuracy: 0.8200
Epoch 169/1000
6400/6400 [==============================] - 0s 55us/sample - loss: 0.0138 - accuracy: 0.9862 

6400/6400 [==============================] - 0s 46us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1729 - val_accuracy: 0.8131
Epoch 218/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1723 - val_accuracy: 0.8144
Epoch 219/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1722 - val_accuracy: 0.8150
Epoch 220/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1720 - val_accuracy: 0.8150
Epoch 221/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1719 - val_accuracy: 0.8144
Epoch 222/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0127 - accuracy: 0.9873 - val_loss: 0.1718 - val_accuracy: 0.8144
Epoch 223/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0127 - accuracy: 0.9873 

...

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1685 - val_accuracy: 0.8181
Epoch 758/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1685 - val_accuracy: 0.8181
Epoch 759/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1685 - val_accuracy: 0.8181
Epoch 760/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1685 - val_accuracy: 0.8181
Epoch 761/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1686 - val_accuracy: 0.8181
Epoch 762/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1686 - val_accuracy: 0.8181
Epoch 763/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 

6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 812/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 813/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 814/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 815/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 816/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1689 - val_accuracy: 0.8188
Epoch 817/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 

6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 866/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 867/1000
6400/6400 [==============================] - 0s 50us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 868/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 869/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 870/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1691 - val_accuracy: 0.8188
Epoch 871/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 

6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 920/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 921/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 922/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 923/1000
6400/6400 [==============================] - 0s 45us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 924/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1693 - val_accuracy: 0.8188
Epoch 925/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 

6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 974/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 975/1000
6400/6400 [==============================] - 0s 49us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 976/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 977/1000
6400/6400 [==============================] - 0s 47us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 978/1000
6400/6400 [==============================] - 0s 48us/sample - loss: 0.0120 - accuracy: 0.9880 - val_loss: 0.1694 - val_accuracy: 0.8188
Epoch 979/1000
6400/6400 [==============================] - 0s 46us/sample - loss: 0.0120 - accuracy: 0.9880 

In [49]:
model.evaluate(x=nn_test_values, y=nn_test_labels, verbose=0)

[0.193884712100029, 0.794]

## Classical Approaches

### Supervised

#### K-Nearest Neigbours

In [5]:
KNN_Data = atlas_processed
KNN_Labels = atlas_labels.to_numpy()

In [6]:
knn_train_values, knn_test_values, knn_train_labels, knn_test_labels = train_test_split(KNN_Data, KNN_Labels, test_size=0.2)

In [7]:
search_max_val = 10

In [8]:
warnings.simplefilter('ignore')
knn_model = KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree')
grid = {"n_neighbors":range(1,search_max_val)}
knn_model_cv = GridSearchCV(knn_model, grid, cv=10, iid=False, n_jobs=-1)
knn_model_cv.fit(KNN_Data, KNN_Labels)
knn_model_cv.best_params_

{'n_neighbors': 8}

In [9]:
knn_model_final = KNeighborsClassifier(n_neighbors=knn_model_cv.best_params_['n_neighbors'], algorithm='ball_tree')
knn_model_final.fit(knn_train_values, knn_train_labels)

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=8, p=2,
                     weights='uniform')

In [10]:
knn_score = knn_model_final.score(knn_test_values, knn_test_labels)

In [11]:
print("Score for KNN is ",knn_score)

Score for KNN is  0.6815


#### Support Vector Machine - SVM

In [12]:
SVM_Data = atlas_processed
SVM_Labels = atlas_labels.to_numpy()

In [13]:
svm_train_values, svm_test_values, svm_train_labels, svm_test_labels = train_test_split(SVM_Data, SVM_Labels, test_size=0.2)

In [14]:
search_max_val = 10

In [15]:
svm_model_lin = SVC(kernel="linear")


In [16]:
grid = {"C":range(1,search_max_val)}

svm_model_cv = GridSearchCV(svm_model_lin, grid, cv=10, iid=False, n_jobs=-1)
svm_model_cv.fit(svm_train_values, svm_train_labels)
svm_model_cv.best_params_

{'C': 5}

In [17]:
svm_model_lin_final = SVC(C=svm_model_cv.best_params_['C'],kernel="linear")
svm_model_lin_final.fit(svm_train_values, svm_train_labels)
svm_lin_score = svm_model_lin_final.score(svm_test_values, svm_test_labels)
print("Score for Linear SVM is ",svm_lin_score)

Score for Linear SVM is  0.733


In [18]:
svm_model_rbf = SVC(kernel="rbf")

In [19]:
grid = {"C":range(1,search_max_val)}

svm_model_cv = GridSearchCV(svm_model_rbf, grid, cv=10, iid=False)
svm_model_cv.fit(svm_train_values, svm_train_labels)
svm_model_cv.best_params_

{'C': 9}

In [21]:
svm_model_rbf_final = SVC(C=svm_model_cv.best_params_['C'],kernel="linear")
svm_model_rbf_final.fit(svm_train_values, svm_train_labels)
svm_rbf_score = svm_model_rbf_final.score(svm_test_values, svm_test_labels)
print("Score for RBF SVM is ",svm_rbf_score)

Score for RBF SVM is  0.733


### Unsupervised

#### K-Means Clustering

In [22]:
KMEANS_Data = atlas_processed
KMEAN_Labels = atlas_labels.to_numpy()

In [23]:
numClusters = 2
iterations = 100

In [26]:
kmeans_model = KMeans(numClusters, max_iter=iterations)

In [27]:
kmeans_model.fit(KMEANS_Data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [44]:
cpy = atlas_labels.to_numpy()
counter = 0
for i in range(0, len(kmeans_model.labels_)):
    if(kmeans_model.labels_[i] == cpy[i]):
        counter += 1
score = counter / len(kmeans_model.labels_)
print(score)

0.5771
